In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split

In [36]:
df = pd.read_csv("labels.csv", header=0)
col_names = 'beagle chihuahua doberman french_bulldog golden_retriever malamute pug saint_bernard scottish_deerhound tibetan_mastiff'.split()


remove_cols = []
for row in range(df.shape[0]):
    if df['breed'].iloc[row] not in col_names:
        remove_cols.append(row)

df = df.drop(remove_cols)
del remove_cols

train_list = os.listdir('train')
test_list = os.listdir('test')
# train_classes = []
train_path = []
train_labels = []
test_path = []
test_labels = []

for row in range(df.shape[0]):
    if (df['id'].iloc[row]+'.jpg') in train_list:
        df['id'].iloc[row] = 'train/'+df['id'].iloc[row]+'.jpg' 
        train_path.append(df['id'].iloc[row])
        train_labels.append(df['breed'].iloc[row])
    elif (df['id'].iloc[row]+'.jpg') in test_list:
        df['id'].iloc[row] = 'test/'+df['id'].iloc[row]+'.jpg'
        test_path.append(df['id'].iloc[row])
        test_labels.append(df['breed'].iloc[row])

image_width = 500
image_length = 375



In [3]:
if(os.path.exists('best_model_augmented.h5')):
    model = load_model('best_model_augmented.h5')
else:
    model = resnet50.ResNet50(weights=None, input_shape=(image_width, image_length, 3), classes=10)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
df_train = pd.DataFrame({'id':train_path, 'breed':train_labels})
df_train.to_excel("TrainData.xlsx")

In [5]:
df_train = pd.read_excel('TrainData.xlsx')

In [6]:
datagen = image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input, horizontal_flip=True, brightness_range=[0.2,1.0], rotation_range=45)
train_df, val_df = train_test_split(df_train, test_size=0.2)

train_generator_df = datagen.flow_from_dataframe(dataframe=train_df, x_col="id", y_col="breed", target_size=(500, 375), batch_size=8)
val_generator_df = datagen.flow_from_dataframe(dataframe=val_df, x_col="id", y_col="breed", target_size=(500, 375), batch_size=8)

print(train_generator_df.class_indices)
print(val_generator_df.class_indices)

Found 672 validated image filenames belonging to 10 classes.
Found 169 validated image filenames belonging to 10 classes.
{'beagle': 0, 'chihuahua': 1, 'doberman': 2, 'french_bulldog': 3, 'golden_retriever': 4, 'malamute': 5, 'pug': 6, 'saint_bernard': 7, 'scottish_deerhound': 8, 'tibetan_mastiff': 9}
{'beagle': 0, 'chihuahua': 1, 'doberman': 2, 'french_bulldog': 3, 'golden_retriever': 4, 'malamute': 5, 'pug': 6, 'saint_bernard': 7, 'scottish_deerhound': 8, 'tibetan_mastiff': 9}


In [7]:

mc = ModelCheckpoint('best_model_augmented.h5',save_best_only=True)

model.fit(x=train_generator_df, validation_data=val_generator_df, epochs=250, batch_size=8, callbacks=[mc])

Epoch 1/250
84/84 [==============================] - 59s 622ms/step - loss: 2.9816 - accuracy: 0.1116 - val_loss: 171.2247 - val_accuracy: 0.0533


D:\PythonEverything\Anaconda\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/250
84/84 [==============================] - 55s 649ms/step - loss: 2.4653 - accuracy: 0.1235 - val_loss: 375.0357 - val_accuracy: 0.0888
Epoch 3/250
84/84 [==============================] - 59s 695ms/step - loss: 2.4028 - accuracy: 0.1176 - val_loss: 2.7110 - val_accuracy: 0.1065
Epoch 4/250
84/84 [==============================] - 61s 729ms/step - loss: 2.4347 - accuracy: 0.1458 - val_loss: 14.5379 - val_accuracy: 0.0769
Epoch 5/250
84/84 [==============================] - 62s 733ms/step - loss: 2.4350 - accuracy: 0.1176 - val_loss: 5.6013 - val_accuracy: 0.0888
Epoch 6/250
84/84 [==============================] - 64s 753ms/step - loss: 2.3715 - accuracy: 0.1205 - val_loss: 35.2763 - val_accuracy: 0.0947
Epoch 7/250
84/84 [==============================] - 65s 768ms/step - loss: 2.3822 - accuracy: 0.1086 - val_loss: 2.4200 - val_accuracy: 0.1657
Epoch 8/250
84/84 [==============================] - 64s 761ms/step - loss: 2.3939 - accuracy: 0.1101 - val_loss: 44.1948 - val_accu

84/84 [==============================] - 67s 797ms/step - loss: 2.1744 - accuracy: 0.2009 - val_loss: 2.1772 - val_accuracy: 0.1834
Epoch 59/250
84/84 [==============================] - 66s 788ms/step - loss: 2.2117 - accuracy: 0.2173 - val_loss: 2.1885 - val_accuracy: 0.1775
Epoch 60/250
84/84 [==============================] - 68s 800ms/step - loss: 2.1932 - accuracy: 0.1949 - val_loss: 2.4928 - val_accuracy: 0.1124
Epoch 61/250
84/84 [==============================] - 65s 776ms/step - loss: 2.1593 - accuracy: 0.2262 - val_loss: 2.2749 - val_accuracy: 0.1716
Epoch 62/250
84/84 [==============================] - 61s 728ms/step - loss: 2.1797 - accuracy: 0.2292 - val_loss: 2.2310 - val_accuracy: 0.1834
Epoch 63/250
84/84 [==============================] - 63s 744ms/step - loss: 2.1615 - accuracy: 0.2351 - val_loss: 4.7236 - val_accuracy: 0.1065
Epoch 64/250
84/84 [==============================] - 65s 766ms/step - loss: 2.1824 - accuracy: 0.2217 - val_loss: 2.6646 - val_accuracy: 0.130

84/84 [==============================] - 65s 778ms/step - loss: 1.9340 - accuracy: 0.3140 - val_loss: 2.2392 - val_accuracy: 0.2367
Epoch 115/250
84/84 [==============================] - 66s 784ms/step - loss: 1.9728 - accuracy: 0.2664 - val_loss: 2.1276 - val_accuracy: 0.2367
Epoch 116/250
84/84 [==============================] - 65s 764ms/step - loss: 1.9538 - accuracy: 0.3021 - val_loss: 2.3759 - val_accuracy: 0.2130
Epoch 117/250
84/84 [==============================] - 64s 763ms/step - loss: 1.9470 - accuracy: 0.2946 - val_loss: 2.3201 - val_accuracy: 0.2071
Epoch 118/250
84/84 [==============================] - 64s 754ms/step - loss: 1.9340 - accuracy: 0.3199 - val_loss: 2.1607 - val_accuracy: 0.2426
Epoch 119/250
84/84 [==============================] - 65s 769ms/step - loss: 1.9038 - accuracy: 0.3185 - val_loss: 2.1047 - val_accuracy: 0.2308
Epoch 120/250
84/84 [==============================] - 65s 772ms/step - loss: 1.9090 - accuracy: 0.3140 - val_loss: 2.1091 - val_accuracy:

84/84 [==============================] - 67s 801ms/step - loss: 1.4190 - accuracy: 0.4985 - val_loss: 2.2275 - val_accuracy: 0.2485
Epoch 171/250
84/84 [==============================] - 67s 796ms/step - loss: 1.3816 - accuracy: 0.5208 - val_loss: 2.1669 - val_accuracy: 0.2663
Epoch 172/250
84/84 [==============================] - 67s 799ms/step - loss: 1.4491 - accuracy: 0.4896 - val_loss: 2.0398 - val_accuracy: 0.3964
Epoch 173/250
84/84 [==============================] - 66s 781ms/step - loss: 1.4296 - accuracy: 0.5030 - val_loss: 2.0228 - val_accuracy: 0.3728
Epoch 174/250
84/84 [==============================] - 70s 825ms/step - loss: 1.3614 - accuracy: 0.5238 - val_loss: 2.3021 - val_accuracy: 0.3846
Epoch 175/250
84/84 [==============================] - 67s 790ms/step - loss: 1.3881 - accuracy: 0.5164 - val_loss: 1.8645 - val_accuracy: 0.3846
Epoch 176/250
84/84 [==============================] - 66s 780ms/step - loss: 1.3498 - accuracy: 0.5387 - val_loss: 2.1888 - val_accuracy:

84/84 [==============================] - 65s 775ms/step - loss: 0.8458 - accuracy: 0.7232 - val_loss: 2.3203 - val_accuracy: 0.4379
Epoch 227/250
84/84 [==============================] - 67s 791ms/step - loss: 0.9203 - accuracy: 0.7024 - val_loss: 1.8877 - val_accuracy: 0.4615
Epoch 228/250
84/84 [==============================] - 66s 788ms/step - loss: 0.9037 - accuracy: 0.6949 - val_loss: 2.0664 - val_accuracy: 0.4201
Epoch 229/250
84/84 [==============================] - 66s 779ms/step - loss: 0.8601 - accuracy: 0.7128 - val_loss: 2.0865 - val_accuracy: 0.4260
Epoch 230/250
84/84 [==============================] - 66s 777ms/step - loss: 0.9153 - accuracy: 0.6860 - val_loss: 2.0747 - val_accuracy: 0.4142
Epoch 231/250
84/84 [==============================] - 66s 787ms/step - loss: 0.9228 - accuracy: 0.6964 - val_loss: 2.2286 - val_accuracy: 0.3669
Epoch 232/250
84/84 [==============================] - 66s 780ms/step - loss: 0.7855 - accuracy: 0.7158 - val_loss: 2.3875 - val_accuracy:

In [8]:
##Overfitted model if trained too much
model = load_model('best_model_augmented.h5')
es = EarlyStopping(monitor='val_accuracy', patience=25)
mc = ModelCheckpoint('best_model_augmented_250.h5',save_best_only=True)
model.fit(x=train_generator_df, validation_data=val_generator_df, epochs=50, batch_size=8, callbacks=[es, mc])

Epoch 1/50
84/84 [==============================] - 53s 625ms/step - loss: 0.6130 - accuracy: 0.7857 - val_loss: 2.7839 - val_accuracy: 0.3491
Epoch 2/50
84/84 [==============================] - 55s 653ms/step - loss: 0.5880 - accuracy: 0.7842 - val_loss: 2.7752 - val_accuracy: 0.3728
Epoch 3/50
84/84 [==============================] - 57s 676ms/step - loss: 0.5621 - accuracy: 0.8244 - val_loss: 2.1103 - val_accuracy: 0.4911
Epoch 4/50
84/84 [==============================] - 65s 776ms/step - loss: 0.5613 - accuracy: 0.8051 - val_loss: 2.6108 - val_accuracy: 0.4320
Epoch 5/50
84/84 [==============================] - 71s 840ms/step - loss: 0.5590 - accuracy: 0.8170 - val_loss: 2.8394 - val_accuracy: 0.3609
Epoch 6/50
84/84 [==============================] - 70s 837ms/step - loss: 0.5571 - accuracy: 0.7932 - val_loss: 2.4205 - val_accuracy: 0.4142
Epoch 7/50
84/84 [==============================] - 77s 919ms/step - loss: 0.6429 - accuracy: 0.7738 - val_loss: 2.3002 - val_accuracy: 0.4024

In [64]:
## Test Data Analysis
## No Labels available for Test Folder Data Analysis
## For Train Data
from sklearn.metrics import roc_auc_score
model = load_model('best_model_augmented.h5')
df = pd.read_excel("TrainData.xlsx")
class_index = {'beagle': 0, 'chihuahua': 1, 'doberman': 2, 'french_bulldog': 3, 'golden_retriever': 4, 'malamute': 5, 'pug': 6, 'saint_bernard': 7, 'scottish_deerhound': 8, 'tibetan_mastiff': 9}
confusion = np.zeros((10,10))
Y_Pred = []
Y_Real = []

for row in range(df.shape[0]):
    img = image.load_img(path=df['id'].iloc[row], target_size=(image_width, image_length))
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    X = resnet50.preprocess_input(X)
    X_Pred = model.predict(X)
    pred_idx = np.argmax(X_Pred)
    real_breed = class_index[df['breed'].iloc[row]]
    classes = np.zeros(10)
    classes[real_breed] = 1
    Y_Real.append(classes)
    Y_Pred.append(X_Pred[0])
    
    confusion[pred_idx][real_breed] += 1
print("Confusion Matrix ",confusion)
print("ROC AUC Score ", roc_auc_score(Y_Real, Y_Pred))

Confusion Matrix  [[35.  3.  1.  4.  2.  0.  0.  2.  0.  0.]
 [10. 25.  5.  5. 13.  4.  7.  0.  3.  2.]
 [ 0.  3. 36.  0.  1.  0.  2.  0.  1.  5.]
 [ 0.  1.  2.  6.  1.  4.  0.  3.  1.  0.]
 [10.  7.  0.  3. 31.  2.  2.  1.  1.  2.]
 [ 0.  4.  0.  2.  3. 15.  6.  2. 16.  2.]
 [20. 17. 10. 29.  7. 40. 58. 13. 31. 14.]
 [19.  3.  4.  7.  1.  5.  2. 51.  1.  1.]
 [ 9.  6.  8. 10.  2.  9. 10.  8. 66.  8.]
 [ 2.  2.  8.  4.  6.  2.  7.  4.  6. 35.]]
ROC AUC Score  0.8264759669191509


In [65]:
scores = {'TP':np.zeros(10), 'TN':np.zeros(10), 'FP':np.zeros(10), 'FN':np.zeros(10), 'Acc':np.zeros(10), 'F1': np.zeros(10)}
for i in range(10):
    scores['TP'][i] = confusion[i][i]
    for row in range(10):
        for col in range(10):
            if(row != col):
                if(row == i):
                    scores['FP'][i] += confusion[row][col]
                elif(col == i):
                    scores['FN'][i] += confusion[row][col]
                else:
                    scores['TN'][i] += confusion[row][col]
                    
    scores['Acc'][i] = (scores['TP'][i]+scores['TN'][i])/(scores['TP'][i]+scores['TN'][i]+scores['FP'][i]+scores['FN'][i])
    precision = scores['TP'][i]/(scores['TP'][i] + scores['FP'][i])
    recall = scores['TP'][i]/(scores['TP'][i] + scores['FN'][i])
    scores['F1'][i] = 2 * (precision * recall)/(precision + recall)
    
print("Accuracy ", scores['Acc'])
print("F1-Score ", scores['F1'])

            

Accuracy  [0.84169884 0.81299213 0.90366089 0.84458078 0.87548638 0.79718876
 0.59889094 0.8576779  0.76320583 0.85521236]
F1-Score  [0.46052632 0.34482759 0.59016393 0.13636364 0.49206349 0.22900763
 0.34834835 0.57303371 0.50381679 0.48275862]


In [68]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast
import base64

app = Flask("DogBreedAPI")
api = Api(app)
class_index = {'beagle': 0, 'chihuahua': 1, 'doberman': 2, 'french_bulldog': 3, 'golden_retriever': 4, 'malamute': 5, 'pug': 6, 'saint_bernard': 7, 'scottish_deerhound': 8, 'tibetan_mastiff': 9}
class Dog(Resource):
    def get(self, image64):
        image = base64.b64decode(image64['image'])
        image_np = np.array(image)
        model.load_model("best_model_augmented.h5")
        pred = model.predict(image.resnet50.preprocess_input)
        class_idx = np.argmax(pred)
        for name in class_index:
            if class_index[name] == class_idx:
                breed = name
                break
        return {'breed':breed ,'score':np.max(pred)}

api.add_resource(Dog, '/Dog')